In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.sql.types import *
from delta import *
from IPython.display import display

In [2]:
builder = SparkSession.builder.appName("olist_demo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.repl.eagerEval.enabled", "true")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/home/fabio/Repos/olist-pyspark-elt-demo/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/fabio/.ivy2/cache
The jars for the packages stored in: /home/fabio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-186f246c-d766-4417-8709-a80c6f6eaf22;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.1 in central
	found io.delta#delta-storage;3.2.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 123ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.1 from central in [default]
	io.delta#delta-storage;3.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

## Config

In [3]:
environment = '' # dev;uat;prd

In [4]:
path = {
    'stg': {'customer': f'../../{environment}/data/stage/olist_customers_dataset.csv'}, 
    'snk': '../../data/bronze/customer'
}

In [5]:
options = {
    "inferSchema": "True",
    "delimiter": ",",
    "header": "True"
}

## Load

In [6]:
df = spark.read.options(**options).format('csv').load(path['stg']['customer'])

In [7]:
print(df)

DataFrame[customer_id: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string]


In [8]:
df_snk = df.select(
    F.col('customer_id').alias('customer_id'),
    F.col('customer_unique_id').alias('customer_unique_id'),
    F.col('customer_zip_code_prefix').alias('customer_zip_code_prefix'),
    F.col('customer_city').alias('customer_city'),
    F.col('customer_state').alias('customer_state')
)

In [9]:
df_snk = df_snk.withColumn("datetime_insert_brz", F.current_date())

## Sink

In [10]:
df_snk.write.format('delta')\
.mode(saveMode='overwrite')\
.save(path['snk'])

24/10/02 18:46:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
